In [1]:
from pylab import *
%matplotlib inline
caffe_root = '/home/deep/develop/caffe/'  # this file should be run from {caffe_root}/examples (otherwise change this line)

import sys
sys.path.insert(0, caffe_root + 'python')
import caffe

# run scripts from caffe root
import os
# os.chdir(caffe_root)

from caffe import layers as L, params as P

def lenet(lmdb, batch_size):
    # our version of LeNet: a series of linear and simple nonlinear transformations
    n = caffe.NetSpec()
    
    n.data, n.label = L.Data(batch_size=batch_size, backend=P.Data.LMDB, source=lmdb,
                             transform_param=dict(scale=1./255), ntop=2)
    
    n.conv1 = L.Convolution(n.data, kernel_size=5, num_output=20, weight_filler=dict(type='xavier'))
    n.pool1 = L.Pooling(n.conv1, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.conv2 = L.Convolution(n.pool1, kernel_size=5, num_output=50, weight_filler=dict(type='xavier'))
    n.pool2 = L.Pooling(n.conv2, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.fc1 =   L.InnerProduct(n.pool2, num_output=500, weight_filler=dict(type='xavier'))
    n.relu1 = L.ReLU(n.fc1, in_place=True)
    n.score = L.InnerProduct(n.relu1, num_output=14, weight_filler=dict(type='xavier'))
    n.loss =  L.SoftmaxWithLoss(n.score, n.label)
    
    return n.to_proto()
    
with open('/home/deep/Documents/zongzuo/pythoncodes/leaders_new/lenet_auto_train.prototxt', 'w') as f:
    f.write(str(lenet('/home/deep/Documents/zongzuo/pythoncodes/leaders_new/leaders_images/politics_train_lmdb', 10)))
    
with open('/home/deep/Documents/zongzuo/pythoncodes/leaders_new/lenet_auto_test.prototxt', 'w') as f:
    f.write(str(lenet('/home/deep/Documents/zongzuo/pythoncodes/leaders_new/leaders_images/politics_val_lmdb', 10)))


In [1]:
!cat ./lenet_auto_solver.prototxt

# The train/test net protocol buffer definition
train_net: "/home/deep/Documents/zongzuo/pythoncodes/leaders_new/lenet_auto_train.prototxt"
test_net: "/home/deep/Documents/zongzuo/pythoncodes/leaders_new/lenet_auto_test.prototxt"
# test_iter specifies how many forward passes the test should carry out.
# In the case of MNIST, we have test batch size 100 and 100 test iterations,
# covering the full 10,000 testing images.
test_iter: 1
# Carry out testing every 500 training iterations.
test_interval: 2
# The base learning rate, momentum and the weight decay of the network.
base_lr: 0.1
momentum: 0.9
weight_decay: 0.0005
# The learning rate policy
lr_policy: "inv"
gamma: 0.0001
power: 0.75
# Display every 100 iterations
display: 1
# The maximum number of iterations
max_iter: 100
# snapshot intermediate results
snapshot: 5
snapshot_prefix: "mnist/lenet"


In [ ]:
import caffe

caffe.set_device(0)
caffe.set_mode_gpu()

### load the solver and create train and test nets
solver = None  # ignore this workaround for lmdb data (can't instantiate two solvers on the same data)
solver = caffe.SGDSolver('/home/deep/Documents/zongzuo/pythoncodes/leaders_new/lenet_auto_solver.prototxt')